# Throttling headers

Service management is an essential component of a stable API Ecosystem.

Basic compontents for service managements are:

- telling your peers when you are not operational, and explicit
  how long it will take to be up and running again;
  
- explicitly telling your peers if they are over quota and
  prevent overquota via throttling headers.
  

While it could be annoying to explicitly state that every response
should contain throttling headers, you can use yaml anchors for that!

```
x-commons:
  throttling-headers: &throttling-headers
    X-RateLimit-Limit:
      $ref: ..
    X-RateLimit-Remaining:
      $ref: ..
    X-RateLimit-Reset:
      $ref: ..

```


## Returning throttling headers

Now we can use the anchor in our `get /echo responses`

```
paths:
  /status
    get:
      ...
      operationId: api.get_status
      responses:
        '200':
          headers:
            <<: *throttling_headers
          ...
```


### Exercise: add throttling headers

Modify the  [ex-07-01-throttling.yaml](/edit/notebooks/oas3/ex-07-01-throttling.yaml) spec so that:

- `get /echo responses` returns the throtting headers in case of `200`;
- ensure that in case of over-quota, a `429` response is returned

In [3]:
### Exercise: throttle requests using the following class

from  oas3.throttling_quota import ThrottlingQuota


quotas = ThrottlingQuota(ttl=30, quota=10)

for i in range(12):
    ret = quotas.add(user=f'user1')
    print(ret)

{'count': 1, 'expires': 1561302810.0, 'quota': 10}
{'count': 2, 'expires': 1561302810.0, 'quota': 10}
{'count': 3, 'expires': 1561302810.0, 'quota': 10}
{'count': 4, 'expires': 1561302810.0, 'quota': 10}
{'count': 5, 'expires': 1561302810.0, 'quota': 10}
{'count': 6, 'expires': 1561302810.0, 'quota': 10}
{'count': 7, 'expires': 1561302810.0, 'quota': 10}
{'count': 8, 'expires': 1561302810.0, 'quota': 10}
{'count': 9, 'expires': 1561302810.0, 'quota': 10}
{'count': 10, 'expires': 1561302810.0, 'quota': 10}
{'count': 11, 'expires': 1561302810.0, 'quota': 10, 'exceeded': True, 'comment': '2019-06-23T15:13:30'}
{'count': 12, 'expires': 1561302810.0, 'quota': 10, 'exceeded': True, 'comment': '2019-06-23T15:13:30'}


Modify [api.py:get_echo](/edit/notebooks/oas3/api.py) such that:

- every authenticated user get its quota;
- if quota is exceeded, `429` is returned;
- on every request, the throttling infos are returned:

  - x-ratelimit-limit: quota
  - x-ratelimit-reset: expires
  - x-ratelimit-remaining: quota - count
  

Now  [run the spec in a terminal](/terminals/1) using

```
cd /code/notebooks/oas3/
connexion run /code/notebooks/oas3/ex-03-02-path.yaml
```

play a bit with the [Swagger UI](https://TODO)

and try making a request!


In [ ]:
!curl http://localhost:5000/datetime/v1/status -kv

### Returning errors.

Our api.get_status implementation always returns `200 OK`, but in the real world APIs could
return different kind of errors.

An interoperable API should:

- fail fast, avoiding that application errors result in stuck connections;
- implement a clean error semantic.

In our Service Management framework we expect that:

- if the Service is unavailable, we must return `503 Service Unavailable` http status
- we must return the `Retry-After` header specifying the number of seconds
  when to retry.

TODO: ADD CIRCUIT_BREAKER IMAGE

To implement this we must:

1. add the returned headers in the OAS3 interface;
2. pass the headers to the flask Response


In [ ]:
from random import randint


def get_status():
    p = randint(1, 5)
    if p == 5:
        return problem(
            status=503,
            title="Service Temporarily Unavailable",
            detail="Retry after the number of seconds specified in the the Retry-After header.",
            headers={'Retry-After': str(p)}
        )
    return problem(
        status=200,
        title="OK",
        detail="So far so good."
    )